In [ ]:
import random
random.seed(3001)

In [ ]:
# input_image_path = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images'
input_image_path = '../input/hotels-50k-eda-and-image-download/hotels-50k/images/train'

In [ ]:
import PIL

def rotate_and_crop(image: PIL.Image, angle_deg: int):
    w_i,h_i = image.size
    
    rotated_expanded_image_with_padding = image.rotate(-angle_deg, expand=True)
    w_p,h_p = rotated_expanded_image_with_padding.size
    
    import math
    angle_rad = np.deg2rad(abs(angle_deg))
    rem_deg_rad = np.deg2rad(90-(abs(angle_deg) % 90))
    
    w_to_remove = int(math.cos(rem_deg_rad)*h_i)
    h_to_remove = int(math.sin(angle_rad)*w_i)
    return rotated_expanded_image_with_padding.crop((w_to_remove,h_to_remove,w_p-w_to_remove,h_p-h_to_remove))

def randomly_rotate_image(image: PIL.Image):
    def do_nothing(image: PIL.Image):
        return image, 0

    def rotate_with_max_degrees(image: PIL.Image, max_degrees: int = 10):
        rotation_deg = random.randrange(-abs(max_degrees), abs(max_degrees) + 1)
        return (rotate_and_crop(image, rotation_deg), rotation_deg)

    def rotate_about_90_degrees_left(image: PIL.Image):
        rotated_90_degrees_left = image.rotate(90, expand=True)
        rotated_image, further_degrees = rotate_with_max_degrees(rotated_90_degrees_left)
        return (rotated_image, -90 + further_degrees)

    def rotate_about_90_degrees_right(image: PIL.Image):
        rotated_90_degrees_right = image.rotate(-90, expand=True)
        rotated_image, further_degrees = rotate_with_max_degrees(rotated_90_degrees_right)
        return (rotated_image, 90 + further_degrees)

    random_image_rotation_fun = np.random.choice(
#         [
# #             do_nothing,
#             rotate_with_max_degrees,
#             rotate_about_90_degrees_left,
#             rotate_about_90_degrees_right
#         ],
#         p=[
# #             0.6,
#             0.5,
#             0.25,
#             0.25
#         ]
        [
            do_nothing,
            rotate_with_max_degrees,
            rotate_about_90_degrees_left,
            rotate_about_90_degrees_right
        ],
        p=[
            0.6,
            0.20,
            0.10,
            0.10
        ]
    )

    return random_image_rotation_fun(image)

In [ ]:
import os
def count_images(path):
    total_images = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            total_images += 1
    return total_images

In [ ]:
print(count_images('../input/hotels-50k-eda-and-image-download'))

In [ ]:
import PIL
import random
import os
import shutil
import numpy as np
import json

if os.path.isdir('data'):
    shutil.rmtree("./data")
os.makedirs('./data/images/train/class')
os.makedirs('./data/images/valid/class')

# n_images_max = 50000#75000#count_images(input_image_path)
# n_training_images = 40000#60000#n_images_m
# 2.7 GB zip

n_images_max = 60000#75000#count_images(input_image_path)
n_training_images = 48000#60000#n_images_max*0.8
# 8.43 GB zip

rotations = []
total_images = 0
for root, dirs, files in os.walk(input_image_path):
    for f in files:
        hotel_image_path = os.path.join(root, f)
        image = PIL.Image.open(hotel_image_path)
        rotated_image, rotation = randomly_rotate_image(image)
        rotations.append(rotation)
        storage_path = './data/images/train/class'
        if total_images >= n_training_images:
            storage_path = './data/images/valid/class'
        rotated_image.save(os.path.join(storage_path, str(total_images)+'.png'))
        total_images += 1
    if total_images > n_images_max:#50000:
        break
print("Total images: {}".format(total_images))

In [ ]:
import json
with open('./data/label.json', 'w') as f:
    json.dump(rotations, f, indent=2)

In [ ]:
# import shutil
# shutil.make_archive('data', 'zip', './data')

In [ ]:
# shutil.rmtree('./data')

In [ ]:
# from IPython.display import FileLink
# FileLink('./data.zip')